In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
with open("user_top_tracks_medium_term.json","r") as f:
    data=json.loads(f.read())


In [3]:
for track in data:
    track['track_id']=track.pop('id')
    track['track_href']=track.pop('href')

Il y a des dictionnaires et des listes imbriqués, notamment pour l'album dans lequel est la musique considérée: la description de cet album contient notamment l'artiste qui a fait l'album, sous forme de liste,qui peut être de taille>1 si il s'agit d'un album collaboratif ou d'un single en featuring. 

Avec la fonction pd.json_normalize on peut extraire les dictionnaires dans ces listes à différents niveaux de profondeur

In [4]:
artists_v2=pd.json_normalize(data,[['album','artists']],meta=['track_id','track_href'])
images=album_images=pd.json_normalize(data,record_path=['album','images'],meta=['track_id','track_href'])
album_album=pd.json_normalize(data,meta=['external_urls','spotify'])
album_artists_v2=pd.json_normalize(data,record_path=['album','artists'],meta=['track_id','track_href'])

On réassemble tout ce que l'on a extrait dans un même tableau,grâce aux clés primaires "track_id" et "track_href".

In [5]:
a=artists_v2.merge(album_artists_v2,how='inner',left_on=['track_id','track_href'],right_on=['track_id','track_href'])

In [6]:
b=a.merge(album_album,how='inner',left_on=['track_id','track_href'],right_on=['track_id','track_href'])

In [7]:
c=b.merge(images,how='inner',left_on=['track_id','track_href'],right_on=['track_id','track_href'])

In [8]:
c.columns

Index(['href_x', 'id_x', 'name_x', 'type_x', 'uri_x',
       'external_urls.spotify_x', 'track_id', 'track_href', 'href_y', 'id_y',
       'name_y', 'type_y', 'uri_y', 'external_urls.spotify_y', 'artists',
       'available_markets', 'disc_number', 'duration_ms', 'explicit',
       'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type',
       'uri', 'album.album_type', 'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.images',
       'album.name', 'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'external_ids.isrc',
       'external_urls.spotify', 'height', 'url', 'width'],
      dtype='object')

In [9]:
#On supprime les colonnes qui contiennent encore des dictionnaires
del c['artists']
del c['album.images']

In [10]:
c.columns

Index(['href_x', 'id_x', 'name_x', 'type_x', 'uri_x',
       'external_urls.spotify_x', 'track_id', 'track_href', 'href_y', 'id_y',
       'name_y', 'type_y', 'uri_y', 'external_urls.spotify_y',
       'available_markets', 'disc_number', 'duration_ms', 'explicit',
       'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type',
       'uri', 'album.album_type', 'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.name',
       'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'external_ids.isrc',
       'external_urls.spotify', 'height', 'url', 'width'],
      dtype='object')

In [11]:
#On renomme de manière plus claire certaines colonnes
d=c.rename(columns={"href_x":'album.artists.href','id_x':'album.artists.id','name_x':'album.artists.name','type_x':'album.artists.type','uri_x':'album.artists.uri',
                 'external_urls.spotify_x':'album.artists.external_urls.spotify','href_y':'artists.href','id_y':'artists.id','name_y':'artists.name','type_y':'artists.type',
                 'uri_y':'artists.uri','external_urls.spotify_y':'artists.external_urls.spotify','height':'images.height','url':'image.url','width':'image.width'})

In [12]:
#Et là on a un tableau (trop) complet qui contient toutes les données.
d.head()

,album.artists.href,album.artists.id,album.artists.name,album.artists.type,album.artists.uri,album.artists.external_urls.spotify,track_id,track_href,artists.href,artists.id,...,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify,images.height,image.url,image.width
0,https://api.spotify.com/v1/artists/69irZ4R8Osr...,69irZ4R8OsrLHjr6VmMmMg,Gaël Faye,artist,spotify:artist:69irZ4R8OsrLHjr6VmMmMg,https://open.spotify.com/artist/69irZ4R8OsrLHj...,4P89bNZVTCmu3zT0IBNLS8,https://api.spotify.com/v1/tracks/4P89bNZVTCmu...,https://api.spotify.com/v1/artists/69irZ4R8Osr...,69irZ4R8OsrLHjr6VmMmMg,...,2020-11-06,day,14,album,spotify:album:42QybsqCIxCc1H2SEskAiv,FRPDK2000080,https://open.spotify.com/track/4P89bNZVTCmu3zT...,640,https://i.scdn.co/image/ab67616d0000b273c841d8...,640
1,https://api.spotify.com/v1/artists/69irZ4R8Osr...,69irZ4R8OsrLHjr6VmMmMg,Gaël Faye,artist,spotify:artist:69irZ4R8OsrLHjr6VmMmMg,https://open.spotify.com/artist/69irZ4R8OsrLHj...,4P89bNZVTCmu3zT0IBNLS8,https://api.spotify.com/v1/tracks/4P89bNZVTCmu...,https://api.spotify.com/v1/artists/69irZ4R8Osr...,69irZ4R8OsrLHjr6VmMmMg,...,2020-11-06,day,14,album,spotify:album:42QybsqCIxCc1H2SEskAiv,FRPDK2000080,https://open.spotify.com/track/4P89bNZVTCmu3zT...,300,https://i.scdn.co/image/ab67616d00001e02c841d8...,300
2,https://api.spotify.com/v1/artists/69irZ4R8Osr...,69irZ4R8OsrLHjr6VmMmMg,Gaël Faye,artist,spotify:artist:69irZ4R8OsrLHjr6VmMmMg,https://open.spotify.com/artist/69irZ4R8OsrLHj...,4P89bNZVTCmu3zT0IBNLS8,https://api.spotify.com/v1/tracks/4P89bNZVTCmu...,https://api.spotify.com/v1/artists/69irZ4R8Osr...,69irZ4R8OsrLHjr6VmMmMg,...,2020-11-06,day,14,album,spotify:album:42QybsqCIxCc1H2SEskAiv,FRPDK2000080,https://open.spotify.com/track/4P89bNZVTCmu3zT...,64,https://i.scdn.co/image/ab67616d00004851c841d8...,64
3,https://api.spotify.com/v1/artists/3ycxRkcZ67A...,3ycxRkcZ67ALN3GQJ57Vig,Masego,artist,spotify:artist:3ycxRkcZ67ALN3GQJ57Vig,https://open.spotify.com/artist/3ycxRkcZ67ALN3...,2Ee9amLUslOwgoJWZEpSSD,https://api.spotify.com/v1/tracks/2Ee9amLUslOw...,https://api.spotify.com/v1/artists/3ycxRkcZ67A...,3ycxRkcZ67ALN3GQJ57Vig,...,2020-11-10,day,1,album,spotify:album:3iJYwx4U2xHyDi7BRjskIJ,USUM72021142,https://open.spotify.com/track/2Ee9amLUslOwgoJ...,640,https://i.scdn.co/image/ab67616d0000b273ce4316...,640
4,https://api.spotify.com/v1/artists/3ycxRkcZ67A...,3ycxRkcZ67ALN3GQJ57Vig,Masego,artist,spotify:artist:3ycxRkcZ67ALN3GQJ57Vig,https://open.spotify.com/artist/3ycxRkcZ67ALN3...,2Ee9amLUslOwgoJWZEpSSD,https://api.spotify.com/v1/tracks/2Ee9amLUslOw...,https://api.spotify.com/v1/artists/3ycxRkcZ67A...,3ycxRkcZ67ALN3GQJ57Vig,...,2020-11-10,day,1,album,spotify:album:3iJYwx4U2xHyDi7BRjskIJ,USUM72021142,https://open.spotify.com/track/2Ee9amLUslOwgoJ...,300,https://i.scdn.co/image/ab67616d00001e02ce4316...,300


In [13]:
#Et maintenant on exporte ce tableau en fichier csv
d.to_csv('user_top_track_medium_term_complet.csv')

In [14]:
#Si on veut avoir un tableau sans les doublons d'artistes par exemple, on peut faire d.drop_duplicates(subset = ['artists.name'])